In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

import matplotlib.pyplot as plt
from typing import Tuple, List
import time
import numpy as np
import sys


,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm
0,27.0,1,156.0,53.9,35.5,69.0,116.0,23.1,13.1,28.0,144.0


In [ ]:
testsize = 0.2

data = {
    'age': [27.0],
    'gender': ['F'],
    'height_cm': [156.0],
    'weight_kg': [53.9],
    'body fat_%': [35.5],
    'diastolic': [69.0],
    'systolic': [116.0],
    'gripForce': [23.1],
    'sit and bend forward_cm': [13.1],
    'sit-ups counts': [28.0],
    'broad jump_cm': [144.0]
}

x = pd.DataFrame(data)
x

mapping = {'M': 0, 'F': 1}
x['gender'] = x['gender'].replace(mapping)
x

In [2]:
dfBody = pd.read_csv('./data/bodyPerformance.csv')

# Define the mapping dictionary
mapping = {'M': 0, 'F': 1}
# Replace values using the mapping dictionary
dfBody['gender'] = dfBody['gender'].replace(mapping)
# Define the mapping dictionary
mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
# Replace values using the mapping dictionary
dfBody['class'] = dfBody['class'].replace(mapping)


# Split the data into a training and test set
X_train, X_test, y_train, y_test = train_test_split(dfBody.drop('class', axis=1), dfBody['class'], test_size=testsize, random_state=42)

# Instantiate a random forests classifier
rf = RandomForestClassifier(random_state=42)

# Define the grid of hyperparameters 'params_rf'
params_rf = {'n_estimators': [100, 350, 500],
                'max_features': ['log2', 'auto', 'sqrt'],
                'min_samples_leaf': [2, 10, 30]}

# Instantiate grid_rf
grid_rf = GridSearchCV(estimator=rf, param_grid=params_rf, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)

# Fit 'grid_rf' to the training set
grid_rf.fit(X_train, y_train)

# Extract best hyperparameters from 'grid_rf'
best_hyperparams = grid_rf.best_params_

print('Best hyerparameters:\n', best_hyperparams)

# Extract best model from 'grid_rf'
best_model = grid_rf.best_estimator_

# Predict the test set labels
y_pred = best_model.predict(X_test)

# Evaluate the test set accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print test set accuracy
print('Accuracy of random forest classifier: {:.3f}'.format(accuracy))


# Predict the test set labels
y_pred = best_model.predict(x)
y_pred

array([2], dtype=int64)

In [3]:
mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}

user_class = mapping[y_pred[0]]
user_class

'C'

In [4]:
problem = "back pain"

In [5]:
# load sources
sources = pd.read_csv('./data/sources_with_exercise_id.csv')
sources

,Problem,Exercise,SourceHead,SourceLink,exercise_id
0,diabetes,Walking,Low impact exercise for diabetes - Diabetes Ca...,https://www.diabetescarecommunity.ca/diet-and-...,0
1,diabetes,Elliptical machine,Low impact exercise for diabetes - Diabetes Ca...,https://www.diabetescarecommunity.ca/diet-and-...,1
2,diabetes,StairMaster,Low impact exercise for diabetes - Diabetes Ca...,https://www.diabetescarecommunity.ca/diet-and-...,2
3,diabetes,Dancing,Get Active! | Diabetes | CDC - Centers for Dis...,https://www.cdc.gov/diabetes/managing/active.html,3
4,diabetes,Swimming,Get Active! | Diabetes | CDC - Centers for Dis...,https://www.cdc.gov/diabetes/managing/active.html,4
...,...,...,...,...,...
67,cardiovascular problems,Swimming,Cardiovascular Exercise - How to Do it Right -...,https://www.verywellfit.com/cardiovascular-exe...,4
68,cardiovascular problems,Rowing,Cardiovascular Exercise - How to Do it Right -...,https://www.verywellfit.com/cardiovascular-exe...,47
69,cardiovascular problems,Cross-country skiing,Cardiovascular Exercise - How to Do it Right -...,https://www.verywellfit.com/cardiovascular-exe...,48
70,cardiovascular problems,Elliptical trainer,Cardiovascular Exercise - How to Do it Right -...,https://www.verywellfit.com/cardiovascular-exe...,49


In [6]:
# load exercise dataset
exercises = pd.read_csv('./data/exercise_dataset_with_exercise_id.csv')
exercises

,"Activity, Exercise or Sport (1 hour)",130 lb,155 lb,180 lb,205 lb,Calories per kg,category,exercise_id
0,"Cycling, mountain bike, bmx",502,598,695,791,1.750730,A,5.0
1,"Cycling, <10 mph, leisure bicycling",236,281,327,372,0.823236,D,5.0
2,"Cycling, >20 mph, racing",944,1126,1308,1489,3.294974,A,5.0
3,"Cycling, 10-11.9 mph, light",354,422,490,558,1.234853,C,5.0
4,"Cycling, 12-13.9 mph, moderate",472,563,654,745,1.647825,B,5.0
...,...,...,...,...,...,...,...,...
243,General cleaning,207,246,286,326,0.721008,D,-1.0
244,"Cleaning, dusting",148,176,204,233,0.515199,D,-1.0
245,Taking out trash,177,211,245,279,0.617427,D,-1.0
246,"Walking, pushing a wheelchair",236,281,327,372,0.823236,D,0.0


In [7]:
# search if sources['Problem'] contains problem
problem = "back pain"
#problem = "asdf"
ex_ids = sources[sources['Problem'].str.lower().str.contains(problem.lower())]
ex_ids
sourceHead = ""
sourceLink = ""

# if ex_ids is not empty then return random ex_id from ex_ids else return -1
import random
if ex_ids.empty:
    ex_id = -1
else:
    ex = ex_ids.sample()
    ex_id = ex['exercise_id'].values[0]
    sourceHead = ex['SourceHead'].values[0]
    sourceLink = ex['SourceLink'].values[0]
ex_id

13

In [8]:
# get exercise from exercises where exercise_id == ex_id
exercise = exercises[exercises['exercise_id'] == ex_id]
exercise

,"Activity, Exercise or Sport (1 hour)",130 lb,155 lb,180 lb,205 lb,Calories per kg,category,exercise_id


In [9]:
# if exercise is not empty return random exercise else return random exercise from exercises
if len(exercise) > 0:
    exercise = exercise.sample()
else:
    exercise = exercises.sample()
exercise

,"Activity, Exercise or Sport (1 hour)",130 lb,155 lb,180 lb,205 lb,Calories per kg,category,exercise_id
189,"Skin diving, moderate",738,880,1022,1163,2.574642,A,-1.0


In [10]:

print(problem, exercise, user_class, sourceHead, sourceLink)

back pain     Activity, Exercise or Sport (1 hour)  130 lb  155 lb  180 lb  205 lb  \
189                Skin diving, moderate     738     880    1022    1163   

     Calories per kg category  exercise_id  
189         2.574642        A         -1.0   C Back exercises in 15 minutes a day - Mayo Clinic https://www.mayoclinic.org/healthy-lifestyle/adult-health/in-depth/back-pain/art-20546859


# LLM

In [ ]:
#!pip install einops
#!pip install bitsandbytes
#!pip install transformers --upgrade
#!pip install torch --upgrade

In [ ]:
import einops
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

In [ ]:
print("Python version")
print (sys.version)
print(transformers.__version__)


print(torch.__version__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device:', device)

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3, 1), 'GB')

In [ ]:
def measure_perf(
    prompt: str, model: AutoModelForCausalLM, tokenizer: AutoTokenizer
) -> Tuple[float, float, torch.Tensor]:
    """
    Measures memory consumption and inference execution time for a given model and prompt.

    Args:
        prompt: Text to be used as input for the model.
        model: Pretrained model used for inference.
        tokenizer: Pretrained tokenizer used to encode the prompt.

    Returns:
        Peak memory consumption in GB, execution time in seconds, and output tensor from the model.
    """
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

    start_time = time.time()

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, max_length=100)

    end_time = time.time()

    peak_mem = torch.cuda.max_memory_allocated()
    peak_mem_consumption = peak_mem / 1e9  # convert bytes to GB

    exec_time = end_time - start_time

    return peak_mem_consumption, exec_time, outputs

In [ ]:
def plot_results(
    mem_consumptions: List[float], execution_times: List[float], dir: str = ""
) -> None:
    """
    Plots memory consumption and execution times.

    Args:
        mem_consumptions: List of memory consumption data in GB.
        execution_times: List of execution time data.
        dir: Destination dir for the plot.
    """
    #os.makedirs(dir, exist_ok=True)

    fig, ax1 = plt.subplots()

    color = "tab:red"
    ax1.set_xlabel("Runs")
    ax1.set_ylabel("GPU Memory Consumption (GB)", color=color)
    ax1.plot(mem_consumptions, color=color)
    ax1.tick_params(axis="y", labelcolor=color)
    ax1.yaxis.get_major_formatter().set_useOffset(False)

    ax2 = ax1.twinx()
    color = "tab:blue"
    ax2.set_ylabel("Execution time (s)", color=color)
    ax2.plot(execution_times, color=color)
    ax2.tick_params(axis="y", labelcolor=color)
    ax2.yaxis.get_major_formatter().set_useOffset(False)

    fig.tight_layout()
    plt.title("GPU Memory Consumption and Execution Time for Each Run")
    fig.subplots_adjust(top=0.88)
    plt.savefig(f"{dir}/falcon_memory_time.png")

In [ ]:
model_path = "tiiuae/falcon-7b-instruct"
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    config=config,
    trust_remote_code=True,
    load_in_4bit=True,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
print(problem, exercise, user_class, sourceHead, sourceLink)

In [ ]:
prompts = [
        "Tell the user in one sentence that " + exercise  + " is a good exercise for " + problem
    ]
mem_consumptions = []
execution_times = []

for i in range(len(prompts)):
    mem_consumption, exec_time, outputs = measure_perf(prompts[i], model, tokenizer)
    mem_consumptions.append(mem_consumption)
    execution_times.append(exec_time)
    print(tokenizer.decode(outputs[0]))

plot_results(mem_consumptions, execution_times)

In [ ]:
# print the model output and add a new line with Source: sourceHead and sourceLink
print(tokenizer.decode(outputs[0]) + "\nSource: " + sourceHead + "\n" + sourceLink)

**Note: Quantization is currently not working, probably because of a bug in bitsandbytes library. Therefor, our hardware is currently not able to run Falcon 7B**